In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import sqlite3

In [3]:
# Адрес сайта, с которого бумем парсить тексты песен
base_url = 'https://www.beesona.ru'

# Адрес сайта, с которого будем парсить бесплатные прокси
proxy_url = 'https://free-proxy-list.net/'

In [4]:
# Функция, которая принимает на вход адрес сайта, которому отправит get-запрос и словарь с прокси (если они уже есть)
# На вход получает адрес сайта и словарь с прокси
# Возвращает html
def get_html(url, proxy=None):
    if proxy == None:
        r = requests.get(url)
    else:
        r = requests.get(url, proxies=proxy)
    return r

### Proxies

In [5]:
# Функция, которая парсит бесплатные прокси.
# На вход получает html
# Возвращает список с прокси. Протокол выбран только http
def get_proxies(html):
    final_proxies_list = []
    soup = BeautifulSoup(html.content, 'html.parser')
    proxies_list = soup.find('tbody').find_all('tr')
    for j in proxies_list:
        proxy_row = j.find_all('td')
        ip = proxy_row[0].text
        port = proxy_row[1].text
        if proxy_row[6].text == 'no':
            protocol = 'http'
            proxy = protocol + '://' + ip + ':' + port
            final_proxies_list.append(proxy)
        else:
            continue
    return final_proxies_list

In [6]:
# Запишим в список proxies_list бесплатные прокси
proxies_list = get_proxies(get_html(proxy_url))

### Songs

In [7]:
# Функция, которая парсит список песен Григория Лепса и ссылки на страницу с текстом.
# На вход получает html
# Возвращает словарь вида: {'Название песни':'url песни'}
def get_songs_dict(html):
    songs_dict = {}
    soup = BeautifulSoup(html.content, 'html.parser')
    songs = soup.find('div', class_='row xls')
    song = songs.find_all('div', class_='col-md-6')
    for i in song:
        song_url = i.find('a', class_='blueBig')
        song_name = song_url.text.split('Григорий Лепс - ')[1]
        songs_dict[song_name] = base_url + song_url.get('href')
    return songs_dict

In [8]:
# Функция для замены косяков в тексте
# На вход получает текст и список косяков
# Возвращает отредактированный текст
def replace_all(text, to_replace):
    for i in to_replace:
        text = text.replace(i, ' ')
    return text

In [9]:
# Функция, которая парсит текст песни
# На вход получает html
# Возвращает текст песни
def get_songs(html):
    soup = BeautifulSoup(html.content, 'html.parser')
    song = soup.find('div', class_='m207').get_text('|', strip=True)
    song = replace_all(song, ['@ Григорий Лепс', '|', '<br/>', '\r\n', '\n', '(2 раза)', '(3раза)', 'Припев.', 'Припев:', 'Взято с сайта http://hm6.ru />', '[Solo]', 'F', 'A#', 'C', 'Dm', 'Gm', 'Музыка Д. Поллыева, Константина Меладзе Слова Константина Меладзе'])   
    return song

In [10]:
# Пройдем циклом по всем страницам с песнями Лепса и соберем словарь dict_of_songs вида {'Название песни': 'url песни'}
dict_of_songs = {}
for page in range (4):
    url = base_url + '/songs/grigoriy_leps/p' + str(page+1) + '/'
    dict_of_songs.update(get_songs_dict(get_html(url, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})))
dict_of_songs

{'Рюмка водки на столе': 'https://www.beesona.ru/songs/grigoriy_leps/ryumka_vodki_na_stole.php',
 'Самый лучший день': 'https://www.beesona.ru/songs/grigoriy_leps/samiy_luchshiy_den.php',
 'Я счастливый': 'https://www.beesona.ru/songs/grigoriy_leps/ya_schastliviy.php',
 'Ну и что': 'https://www.beesona.ru/songs/grigoriy_leps/nu_i_chto.php',
 'Она не твоя': 'https://www.beesona.ru/songs/grigoriy_leps/ona_ne_tvoya.php',
 'Спокойной ночи, господа': 'https://www.beesona.ru/songs/grigoriy_leps/47163.php',
 'Прости': 'https://www.beesona.ru/songs/grigoriy_leps/10025.php',
 'Водопадом': 'https://www.beesona.ru/songs/grigoriy_leps/vodopadom.php',
 'Натали': 'https://www.beesona.ru/songs/grigoriy_leps/natali.php',
 'Опять метель': 'https://www.beesona.ru/songs/grigoriy_leps/9690.php',
 'Вьюга': 'https://www.beesona.ru/songs/grigoriy_leps/4530.php',
 'Я тебя не люблю': 'https://www.beesona.ru/songs/grigoriy_leps/ya_tebya_ne_lyublyu.php',
 'Уходи красиво': 'https://www.beesona.ru/songs/grigoriy_l

In [11]:
# Пройдем циклом по словарю dict_of_songs и соберем словарь dict_of_song_texts вида {'Название песни': ['url песни', 'Текст песни']}
dict_of_song_texts = {}
for key in dict_of_songs:
    dict_of_song_texts[key] = [dict_of_songs[key], get_songs(get_html(dict_of_songs[key], {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]}))]
dict_of_song_texts

{'Рюмка водки на столе': ['https://www.beesona.ru/songs/grigoriy_leps/ryumka_vodki_na_stole.php',
  'Ночь по улицам пошла звездной поступью царицы. Слов и чисел простота у небесного моста. Раскидала перья птица. Не забудутся ни кем праздник губ, обиды глаз. Забери меня в свой плен. Эту линию колен целовать последний раз.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики. Не легко тебя отдать парусам ветров и птиц. Может, даже, не понять, может, даже, не узнать среди тысяч женских лиц. Пусть глаза мои молчат. Молча смотрят на луну. Если кто поймает взгляд, поторопится назад, сам не знаю почему.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики.  '],
 'Самый лучший день': ['https://www.beesona.ru/songs/grigoriy_leps/samiy_luchshiy_den.php',
  'Триста тысяч часов за спин

In [20]:
dict_of_song_texts.keys()

dict_keys(['Рюмка водки на столе', 'Самый лучший день', 'Я счастливый', 'Ну и что', 'Она не твоя', 'Спокойной ночи, господа', 'Прости', 'Водопадом', 'Натали', 'Опять метель', 'Вьюга', 'Я тебя не люблю', 'Уходи красиво', 'Лондон', 'Что может человек', 'Лабиринт', 'Парус', 'Крыса ревность', 'Берега', 'Ты Пришла', 'Чёрная Кошка', 'Купола', 'Зачем тебе я?', 'Настоящая женщина', 'Спасибо, ребята', 'Она', 'Мне одиноко', 'Песня о друге', 'Научись Летать', 'ВОДОПАД', 'Целая жизнь', 'Храни вас Бог', 'Расскажи', 'Я Верну Тебя', 'Ты опоздала', 'Где-то за тучами', 'Капитан Арктика', 'Я слушал дождь', 'Бессонница', 'Беги по небу', 'Падают листья', 'В городе дождь', 'Ангел Ушёл В Запой', 'Шелест', 'Московская Песня', 'Танго разбитых сердец', 'Зима', 'Романс', 'Боже, как долго', 'Если Хочешь-Уходи', 'Новогодняя', 'Дом хрустальный', 'Чижик', 'Выбрось Из Головы', 'Королева', 'Клуб Одиноких Сердец......', 'Дом', 'Странная Любовь', 'Господи, Дай Мне Сил', 'Игрок', 'Спасите наши души', 'Гололед', 'Двое у 

### Data base

In [12]:
con = sqlite3.connect('song.db') # создает БД

In [13]:
cur = con.cursor() 
cur.execute('DROP TABLE text_songs')
cur.execute('CREATE TABLE text_songs(author, song_name, url, text)') # Я добавил еще столбец с названием песни
for key in dict_of_song_texts:
    sql_insert = """
    INSERT INTO text_songs (author, song_name, url, text)
    VALUES ('{}', '{}', '{}', '{}');
    """.format('Григорий Лепс', key, dict_of_song_texts[key][0], dict_of_song_texts[key][1])    
    res = cur.execute(sql_insert)
    con.commit()

In [14]:
res = cur.execute("Select * from text_songs")
result = res.fetchall()

In [15]:
len(result)

111

In [16]:
result

[('Григорий Лепс',
  'Рюмка водки на столе',
  'https://www.beesona.ru/songs/grigoriy_leps/ryumka_vodki_na_stole.php',
  'Ночь по улицам пошла звездной поступью царицы. Слов и чисел простота у небесного моста. Раскидала перья птица. Не забудутся ни кем праздник губ, обиды глаз. Забери меня в свой плен. Эту линию колен целовать последний раз.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики. Не легко тебя отдать парусам ветров и птиц. Может, даже, не понять, может, даже, не узнать среди тысяч женских лиц. Пусть глаза мои молчат. Молча смотрят на луну. Если кто поймает взгляд, поторопится назад, сам не знаю почему.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики.   Только рюмка водки на столе, ветер плачет за окном. Тихо болью отзываются во мне этой молодой луны крики.  '),
 ('Григорий Лепс',
  'Самый лучший день',
  'https://www.beesona.ru/songs/grigoriy_leps/samiy_luchsh